In [1]:
import transformers
import datasets 
import peft

In [327]:
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, DatasetDict

lora_config = LoraConfig(
    r=4, # number of the parameters to train.
    lora_alpha=1, # magnitude of the weight matrix
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type="SEQ_CLS" # sequence classification
)

In [329]:
from transformers import BertTokenizer, BertModel
model_checkpoint = "google-bert/bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint, use_fast=True)
foundation_model = BertModel.from_pretrained(model_checkpoint)

In [330]:
import pandas as pd

DATA_PATH = "/home/jovyan/work/ULM-25-authorship-profiling/data/"

df_train = pd.read_csv(DATA_PATH + "data_train.csv")
df_test = pd.read_csv(DATA_PATH + "data_test.csv")
df_val = pd.read_csv(DATA_PATH + "data_val.csv")

In [332]:
dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(df_train).select(range(10000))
dataset['validation'] = Dataset.from_pandas(df_val).select(range(1000))
dataset['test'] = Dataset.from_pandas(df_test).select(range(1000))

In [333]:
dataset = dataset.filter(lambda example: 
                         example["text"] is not None and 
                         example["gender"] is not None and
                         example ["age"] is not None)
dataset

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 1000
    })
})

In [334]:
from transformers import AutoTokenizer, AutoModel, PreTrainedModel, PretrainedConfig
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import numpy as np

age_encoder = LabelEncoder()
age_encoder.fit(df_train["age"].tolist() + df_val["age"].tolist() + df_test["age"].tolist())
num_age_labels = len(age_encoder.classes_)

def preprocess_function_dict(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

    age_labels = age_encoder.transform(examples["age"])
    gender_labels = [
        {"male": 0, "female": 1}[label] for label in examples["gender"]
    ]

    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "age_labels": age_labels,
        "gender_labels": gender_labels,
        "labels": np.stack([age_labels, gender_labels], axis=1)
    }

In [335]:
tokenized_dataset = dataset.map(
    preprocess_function_dict,
    batched=True,
    remove_columns=['text', 'age', 'gender']
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [336]:
class JointClassificationConfig(PretrainedConfig):
    def __init__(self, num_age_labels=None, num_gender_labels=None, loss_alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.num_age_labels = num_age_labels
        self.num_gender_labels = num_gender_labels

In [337]:
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import CrossEntropyLoss

class BertForJointClassification(PreTrainedModel):
    config_class = JointClassificationConfig
    
    def __init__(self, config, model):
        super().__init__(config)
        self.num_age_labels = config.num_age_labels
        self.num_gender_labels = config.num_gender_labels

        self.bert = model
        self.age_classifier = nn.Linear(config.hidden_size, self.num_age_labels)
        self.gender_classifier = nn.Linear(config.hidden_size, self.num_gender_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.age_loss_fct = CrossEntropyLoss()
        self.gender_loss_fct = CrossEntropyLoss()
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        age_labels=None,
        gender_labels=None,
        **kwargs
    ):
        # get BERT outputs
        x = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        # pool outputs to get sequence representation
        x = x.last_hidden_state[:, 0]
        x = self.dropout(x)
        # get logits from task head
        age_logits = self.age_classifier(x)
        gender_logits = self.gender_classifier(x)
        loss_age = self.age_loss_fct(age_logits, age_labels)
        loss_gender = self.gender_loss_fct(gender_logits, gender_labels)
        return SequenceClassifierOutput(loss={"loss_age": loss_age, "loss_gender": loss_gender}, 
                                        logits=torch.cat([age_logits, gender_logits], dim=1))

In [338]:
def setup_model(dataset, model_name="bert-base-uncased", peft=True):
    config = JointClassificationConfig(
        name_or_path=model_name,
        num_age_labels=num_age_labels,
        num_gender_labels=2,
        hidden_size=768,
        hidden_dropout_prob=0.1,
    )
    bert = BertForJointClassification(config, foundation_model)
    if peft:
        bert = get_peft_model(bert, lora_config)
    return bert, config

In [345]:
# optional
peft_model.unload()

BertForJointClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [346]:
peft_model, config = setup_model(dataset, model_checkpoint)

In [347]:
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    age_labels, gender_labels = eval_pred.label_ids

    age_logits = logits[:, :num_age_labels]
    gender_logits = logits[:, num_age_labels:]

    age_preds = np.argmax(age_logits, axis=-1)
    gender_preds = np.argmax(gender_logits, axis=-1)

    age_acc = (age_preds == age_labels).mean()
    gender_acc = (gender_preds == gender_labels).mean()
    joint_acc = np.mean((age_preds == age_labels) & (gender_preds == gender_labels))

    return {"age_acc": age_acc, "gender_acc": gender_acc, "joint_acc": joint_acc}

In [349]:
from typing import Dict, Union, Any

class TrainerWithCustomLoss(Trainer):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.age_alpha = 0.5
        self.gender_alpha = 1 - self.age_alpha
    
    def training_step(
        self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]], num_items_in_batch=None
    ) -> torch.Tensor:
        model.train()
        if hasattr(self.optimizer, "train") and callable(self.optimizer.train):
            self.optimizer.train()
        inputs = self._prepare_inputs(inputs)
        
        with self.compute_loss_context_manager():
            loss = self.compute_loss(model, inputs, num_items_in_batch=num_items_in_batch)
        self.accelerator.backward(loss)
        
        # Finally we need to normalize the loss for reporting
        if num_items_in_batch is None:
            return loss.detach() / self.args.gradient_accumulation_steps
        return loss.detach()

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)
        loss_age, loss_gender = outputs.loss["loss_age"], outputs.loss["loss_gender"]
        loss = self.age_alpha * loss_age + self.gender_alpha * loss_gender
        if return_outputs:
            return loss, outputs
        return loss

In [350]:
from transformers import Trainer, TrainingArguments, default_data_collator

print(peft_model.print_trainable_parameters())
log_frequency = 500

training_args = TrainingArguments(
    "bert-lora-for-author-profiling",
    auto_find_batch_size=True,
    num_train_epochs=10,
    eval_strategy="steps",   
    logging_strategy="steps",
    logging_steps=log_frequency,
    save_strategy="no",
    learning_rate=5e-5,
    report_to="wandb"
)

trainer = TrainerWithCustomLoss(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

trainable params: 187,420 || all params: 109,672,760 || trainable%: 0.1709
None


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Age Acc,Gender Acc,Joint Acc
500,1.770400,1.709663,0.112000,0.596000,0.058000
1000,1.708600,1.691807,0.116000,0.624000,0.068000
